<a href="https://colab.research.google.com/github/zhongjie-wu/579project/blob/main/debugging_cen_nash_dqn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
# ! pip install pettingzoo[mpe]
# ! pip install tf-agents

In [104]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from pettingzoo.mpe import simple_speaker_listener_v3, simple_reference_v2, simple_world_comm_v2
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import tensorflow as tf
from collections import deque
import time
import random
from tqdm import tqdm
import os
import json
import matplotlib.pyplot as plt
import tf_agents.utils.nest_utils as nest_utils
from tf_agents.specs import tensor_spec
# from tf_agents.replay_buffers import py_uniform_replay_buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer

In [105]:
#Global Variables
DISCOUNT = 0.99
MINIBATCH_SIZE = 32  # How many steps (samples) to use for training
UPDATE_TARGET_EVERY = 5  # Terminal states (end of episodes)

# Environments
EPISODES = 500
MAX_CYCLES = 25

REPLAY_MEMORY_SIZE = 100  # How many last steps to keep for model training
CRITIC_MIN_REPLAY_MEMORY_SIZE = 50
AGENT_MIN_REPLAY_MEMORY_SIZE = 80  # Minimum number of steps in a memory to start training

# Exploration settings
EPSILON = 0.1  # decaying epsilon
EPSILON_DECAY = 0.99975
MIN_EPSILON = 0.001

## Nash DQN

#### Replay Memory

The reason why this is implemented as a seperate class is because the data in this memory will be shared across the listener, speaker and centralized DQN network. Hence, saving into one object saves memory at runtime (i.e. no multiple appending).

In [106]:
class ReplayMemory:
    def __init__(self, max_len = REPLAY_MEMORY_SIZE):
        self.data_spec =  (
            tf.TensorSpec([3,], tf.float32, 'S_sp'),
            tf.TensorSpec([11,], tf.float32, 'S_ls'),
            tf.TensorSpec([], tf.float32, 'A_sp'),
            tf.TensorSpec([], tf.float32, 'A_ls'),
            tf.TensorSpec([], tf.float32, 'R_sp'),
            tf.TensorSpec([], tf.float32, 'R_ls'),
            tf.TensorSpec([3,], tf.float32, 'S_next_sp'),
            tf.TensorSpec([11,], tf.float32, 'S_next_ls'),
            tf.TensorSpec([], tf.bool, 'done_sp'),
            tf.TensorSpec([], tf.bool, 'done_ls'),
        )
        
        # self.data_spec =  (
        #     tf.TensorSpec((3,), np.float32, 'S_sp'),
        #     tf.TensorSpec((11,), np.float32, 'S_ls'),
        #     tf.TensorSpec((3,), np.float32, 'A_sp'),
        #     tf.TensorSpec((5,), np.float32, 'A_ls'),
        #     tf.TensorSpec((1,), np.float32, 'R_sp'),
        #     tf.TensorSpec((1,), np.float32, 'R_ls'),
        #     tf.TensorSpec((3,), np.float32, 'S_next_sp'),
        #     tf.TensorSpec((11,),np.float32, 'S_next_ls'),
        #     tf.TensorSpec((1,), np.bool_, 'done_sp'),
        #     tf.TensorSpec((1,), np.bool_, 'done_ls'),
        # )

        self.size = 0

        # self.replay_memory = deque(maxlen=max_len)
        self.replay_memory = tf_uniform_replay_buffer.TFUniformReplayBuffer(
                                data_spec=self.data_spec,
                                max_length=max_len,
                                batch_size=MINIBATCH_SIZE
                                )

    def add_sample(self, sample):
        # sp = speaker, ls = listener, the format of a sample is:
        # (S_sp, S_ls, A_sp, A_ls, R_sp, R_ls, S_next_sp, S_next_ls, done_sp, done_ls)
        # self.replay_memory.append(sample)
        self.replay_memory.add_batch(sample)
        if self.size < REPLAY_MEMORY_SIZE:
            self.size += 1


    def get_size(self):
        return self.size
    
    def sample_minibatch(self, minibatch_size = MINIBATCH_SIZE):
        # return random.sample(self.replay_memory, minibatch_size)
        return self.replay_memory.get_next(sample_batch_size=minibatch_size)
    
    def get_mem(self):
        return self.replay_memory

#### Nash DQN agent (For each game env agent)

In [107]:
class NashDQNAgent:
    def __init__(self, input_layer_size, action_space_size, ReplayMemoryObject, is_speaker, critic):
        # Main model which we use to train
        self.model = self.create_model(input_layer_size, action_space_size)

        # Target network to make sure the updating is stable
        self.target_model = self.create_model(input_layer_size, action_space_size)
        self.target_model.set_weights(self.model.get_weights())

        # The array to keep the memory for the last n steps for training
        self.replay_memory = ReplayMemoryObject

        # Count when to update target network with main network's weights
        self.target_update_counter = 0

        # Check if this agent is a speaker, if not then listener
        self.is_speaker = is_speaker

        # Add the critic -- a centalized network to give Q values for joint actions by inputing joint observations
        self.critic = critic

    def create_model(self, input_layer_size, action_space_size):
        model = Sequential()
        model.add(Dense(64, activation='relu', input_shape=(input_layer_size,)))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(action_space_size, activation = 'linear'))
        model.compile(loss="mse", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
        return model

    def train(self, terminal_state):
        # Start training only if enough transition samples has been collected in the memory
        if self.replay_memory.get_size() < AGENT_MIN_REPLAY_MEMORY_SIZE:
            return

        # Get a minibatch from memory replay table
        minibatch = self.replay_memory.sample_minibatch(minibatch_size = MINIBATCH_SIZE)

        # Get the current states and their corresponding q values for each sample in the minibatch
        current_states = np.array([transition[0] for transition in minibatch]) if self.is_speaker else np.array([transition[1] for transition in minibatch])
        current_qs_list = self.model.predict(current_states, verbose=0)

        X = []
        y = []

        # for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
        for index, (S_sp, S_ls, A_sp, A_ls, R_sp, R_ls, S_next_sp, S_next_ls, done_sp, done_ls) in enumerate(minibatch):
            done = done_sp if self.is_speaker else done_ls
            reward = R_sp if self.is_speaker else R_ls
            action = A_sp if self.is_speaker else A_ls
            current_state = S_sp if self.is_speaker else S_ls

            if not done:
                # Calculate Nash Q using the centralized network
                joint_observation = np.concatenate((S_next_sp, S_next_ls), axis=None)
                joint_q_vals = self.critic.get_qs(joint_observation)
                nash_q = np.max(joint_q_vals)
                # Nash update
                new_q = reward + DISCOUNT * nash_q 
            else:
                new_q = reward
            
            # Update Q value for the given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # Prepare training data
            X.append(current_state)
            y.append(current_qs)

        self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, shuffle=False, verbose=0)

        if terminal_state:
            self.target_update_counter += 1
        
        # update target network with weights of main network if condition satisfied
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape), verbose=0)[0]

#### Critic DQN Agent (For providing the Nash Q value of joint states/actions)

This DQN Agent serves as the critic for our Nash DQN algorithm. It takes in the joint states observed by the two agents, and then output an array of Q_valus that corresponds to each combination of agents'action. In a fully collaborative settings, we know that for a given state, the joint actions that lead to the maximal Q value is the nash equilibria move and this maximal Q value is the Nash_Q value.

In [108]:
class JointCritic:
    def __init__(self, input_layer_size, action_space_size, ReplayMemoryObject):
        # Main model which we use to train
        self.model = self.create_model(input_layer_size, action_space_size)

        # Target network to make sure the updating is stable
        self.target_model = self.create_model(input_layer_size, action_space_size)
        self.target_model.set_weights(self.model.get_weights())

        # The array to keep the memory for the last n steps for training
        self.replay_memory = ReplayMemoryObject

        # Count when to update target network with main network's weights
        self.target_update_counter = 0


    def create_model(self, input_layer_size, action_space_size):
        model = Sequential()
        model.add(Dense(64, activation='relu', input_shape=(input_layer_size,)))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(action_space_size, activation = 'linear'))
        model.compile(loss="mse", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
        return model
    
    def train(self, terminal_state):
        # Start training only if enough transition samples has been collected in the memory
        if self.replay_memory.get_size() < CRITIC_MIN_REPLAY_MEMORY_SIZE:
            return

        # Get a minibatch from memory replay table
        minibatch = self.replay_memory.sample_minibatch(minibatch_size = MINIBATCH_SIZE)

        print(minibatch[8])
        print(type(minibatch[8]))

        # Get the current states and their corresponding q values for each sample in the minibatch
        current_states = np.array([np.concatenate((transition[0], transition[1]), axis=None) for transition in minibatch])
        # current_states = StandardScaler().fit_transform(current_states)
        current_qs_list = self.model.predict(current_states, verbose=0)

        # Get the next states their corresponding q values for each sample in the minibatch
        new_current_states = np.array([np.concatenate((transition[6], transition[7]), axis=None) for transition in minibatch])
        future_qs_list = self.target_model.predict(new_current_states, verbose=0)

        X = []
        y = []

        # for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
        for index, (S_sp, S_ls, A_sp, A_ls, R_sp, R_ls, S_next_sp, S_next_ls, done_sp, done_ls) in enumerate(minibatch):
            done = done_sp or done_ls
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = R_sp + R_ls + DISCOUNT * max_future_q
            else:
                new_q = R_sp + R_ls
            
            # Update Q value for the given state
            current_qs = current_qs_list[index]
            action_idx = np.ravel_multi_index((A_sp, A_ls), dims=(3, 5))
            current_qs[action_idx] = new_q

            # Prepare training data
            X.append(current_states[index])
            y.append(current_qs)

        self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, shuffle=False, verbose=0)

        if terminal_state:
            self.target_update_counter += 1
        
        # update target network with weights of main network if condition satisfied
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape), verbose=0)[0]

#### Experiments

In [109]:
def eps_greedy_act_selection(epsilon, action_space_size, q_values):
    if np.random.random() < epsilon:
        # randomly choose one action
        return np.random.randint(0, action_space_size)
    else:
        # all q values
        return np.argmax(q_values)

In [110]:
AGENT_NAMES = ['speaker_0', 'listener_0']
AGENT_INFOS = {name: {"agent_idx": 0 if name == 'speaker_0' else 1,
                        "action_space_size": 3 if name == 'speaker_0' else 5,
                        "input_layer_size": 3 if name == 'speaker_0' else 11,
                        "is_speaker": True if name == 'speaker_0' else False
                        } for name in AGENT_NAMES}

UPDATE_COUNTER = 0
ALL_REWARDS = {agent_name:[] for agent_name in AGENT_NAMES}
epsilon = EPSILON

# Create a replay buffer
replay_buff = ReplayMemory(max_len = REPLAY_MEMORY_SIZE)

# Create the critic DQN Agent
critic_dqn = JointCritic(input_layer_size=14, action_space_size=15, ReplayMemoryObject=replay_buff)

# Create the Nash DQN Agents
NASH_DQN_AGENTS = {name: NashDQNAgent(input_layer_size=AGENT_INFOS[name]["input_layer_size"],
                                      action_space_size=AGENT_INFOS[name]["action_space_size"],
                                      ReplayMemoryObject=replay_buff,
                                      is_speaker=AGENT_INFOS[name]["is_speaker"],
                                      critic=critic_dqn) for name in AGENT_NAMES}

# Create the environment
env = simple_speaker_listener_v3.env(max_cycles=MAX_CYCLES, continuous_actions=False)

for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):
    # Reset the environment and the reward for this new episode
    env.reset()
    episode_agent_reward = {agent_name:0 for agent_name in AGENT_NAMES}

    # Initialize the SARSD for collecting and building dataset later
    S_sp=S_ls=A_sp=A_ls=R_sp=R_ls=S_next_sp=S_next_ls=done_sp=done_ls=None

    for agent in env.agent_iter():
        if env.truncations[agent] == True or env.terminations[agent] == True:
                env.step(None)
                continue
        
        # Get the current agent
        nash_dqn_agent = NASH_DQN_AGENTS[agent]
        # Observe the current state
        state_curr = env.observe(agent)
        # Get the Q values for each action of the curr state
        q_state_curr = nash_dqn_agent.get_qs(state_curr)
        # Choose and take an action
        action = eps_greedy_act_selection(epsilon, AGENT_INFOS[agent]["action_space_size"], q_state_curr)
        env.step(action)
        # Get reward and accumulate it
        _, reward, termination, truncation, info = env.last()
        episode_agent_reward[agent] += reward
        # Observe the next state
        state_next = env.observe(agent)

        # Update the variables
        if nash_dqn_agent.is_speaker:
            S_sp = state_curr
            A_sp = action
            R_sp = reward
            S_next_sp = state_next
            done_sp = termination or truncation
        else:
            S_ls = state_curr
            A_ls = action
            R_ls = reward
            S_next_ls = state_next
            done_ls = termination or truncation

        # Increase the update counter
        UPDATE_COUNTER += 1
        
        # Add this sample to the replay buffer after each agent takes a move
        if UPDATE_COUNTER == 2:
            UPDATE_COUNTER = 0
            transition = (S_sp, S_ls, A_sp, A_ls, R_sp, R_ls, S_next_sp, S_next_ls, done_sp, done_ls)
            replay_buff.add_sample(transition)

            # Clean up after adding
            S_sp=S_ls=A_sp=A_ls=R_sp=R_ls=S_next_sp=S_next_ls=done_sp=done_ls=None

            # Trains the centralized critic
            # critic_dqn.train(done_sp or done_ls)
        
        # Trains the agent if has enough data
        # nash_dqn_agent.train(termination or truncation)

    # store the total rewards for last game play in one episode
    # for name in AGENT_NAMES:
        # ALL_REWARDS[name].append(episode_agent_reward[name]/MAX_CYCLES)

    # Peform epsilon decay
    if epsilon > MIN_EPSILON:
            epsilon *= EPSILON_DECAY
            epsilon = max(MIN_EPSILON, epsilon)


# # Finally, plot the average reward per step per episode per agent
# plt.plot(range(EPISODES), ALL_REWARDS['speaker_0'])
# plt.title('Avg reward per step for speaker_0')
# plt.xlabel('num_episodes')
# plt.ylabel('reward')
# plt.show()

# plt.plot(range(EPISODES), ALL_REWARDS['listener_0'])
# plt.title('Avg reward per step for listener_0')
# plt.xlabel('num_episodes')
# plt.ylabel('reward')
# plt.show()

  0%|          | 0/500 [00:00<?, ?episodes/s]


InvalidArgumentError: ignored